In [1]:
import sys
sys.path.append("/home/ubuntu/MultiModalDeepFake")
import nemo.collections.asr as nemo_asr 
import pandas as pd

/home/ubuntu/.local/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.24.3) or chardet (5.1.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-05-10 19:56:59.386353: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 19:56:59.491372: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-10 19:57:00.030926: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.c

In [2]:
from packages.LJDataLoader import LJDataLoader
from packages.AudioEmbeddingsManager import AudioEmbeddingsManager
from packages.ModelManager import ModelManager
from packages.CadenceModelManager import CadenceModelManager
import packages.AnalysisManager as am
from packages.SmileFeatureManager import SmileFeatureManager

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



## Generate the Train/Dev/Test

In [3]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
loader = LJDataLoader(data_path=file_path)
loader.sample(0.1)
loader.splitData()

source_architectures = ['Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge', 'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow']
new_col_name = 'RandWaveFake'
loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)

#data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=['RandWaveFake', 'ElevenLabs', 'UberDuck'])
data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=['ElevenLabs'])

In [4]:
train_count = data_df[data_df['type'] == 'train'].shape[0]
dev_count = data_df[data_df['type'] == 'dev'].shape[0]
test_count = data_df[data_df['type'] == 'test'].shape[0]

print(f'# of Train instances: {train_count}')
print(f'# of Dev instances: {dev_count}')
print(f'# of Test instances: {test_count}')

# of Train instances: 1566
# of Dev instances: 522
# of Test instances: 522


In [5]:
data_df.head()

,type,id,architecture,path,label,multiclass_label
0,train,LJ002-0156,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
1,train,LJ009-0241,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
2,train,LJ037-0241,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
3,train,LJ028-0476,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
4,train,LJ016-0016,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0


In [6]:
data_df = pd.read_csv('/home/ubuntu/data/demo/demo_train_test_split.csv')

In [7]:
data_df

,id,architecture,path,label,multiclass_label,type
0,206A-lecture01-03_clip63,Real,/home/ubuntu/data/demo/hany_206b_clip_audio_16...,0,0,train
1,206A-lecture01-05_clip30,Real,/home/ubuntu/data/demo/hany_206b_clip_audio_16...,0,0,train
2,LJ028-0215,ElevenLabs,/home/ubuntu/data/demo/hany_elevenlabs_16KHz/L...,1,1,train
3,LJ011-0030,ElevenLabs,/home/ubuntu/data/demo/hany_elevenlabs_16KHz/L...,1,1,train
4,206A-lecture02-02_clip25,Real,/home/ubuntu/data/demo/hany_206b_clip_audio_16...,0,0,train
...,...,...,...,...,...,...
1223,206A-lecture01-03_clip107,Real,/home/ubuntu/data/demo/hany_206b_clip_audio_16...,0,0,test
1224,LJ003-0300,ElevenLabs,/home/ubuntu/data/demo/hany_elevenlabs_16KHz/L...,1,1,test
1225,LJ031-0218,ElevenLabs,/home/ubuntu/data/demo/hany_elevenlabs_16KHz/L...,1,1,test
1226,LJ022-0023,ElevenLabs,/home/ubuntu/data/demo/hany_elevenlabs_16KHz/L...,1,1,test


## Generate the Features

In [8]:

speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained(model_name='titanet_large')
embedding_manager = AudioEmbeddingsManager(model=speaker_model, data=data_df)
em_feature_df, em_feature_cols = embedding_manager.generateFeatureDf()


[NeMo I 2023-05-10 19:57:04 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-05-10 19:57:04 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-05-10 19:57:04 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-05-10 19:57:05 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-05-10 19:57:05 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-05-10 19:57:05 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-05-10 19:57:05 features:267] PADDING: 16
[NeMo I 2023-05-10 19:57:05 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 2/2 [00:00<00:00,  6.66it/s]


<font color='red'> Sarah to update

In [9]:

cadence_manager = CadenceModelManager(data_df)
cad_feature_df, cad_feature_cols, scalar =  cadence_manager.run_cadence_feature_extraction_pipeline() # Add param for load features or not
# TO DO: save the features down to CSV so can be loaded with features loader 
print(type(scalar))

Normalizing amplitudes
Truncating silences
Truncating audio 100/1228 (8%)
Truncating audio 200/1228 (16%)
Truncating audio 300/1228 (24%)
Truncating audio 400/1228 (33%)
Truncating audio 500/1228 (41%)
Truncating audio 600/1228 (49%)
Truncating audio 700/1228 (57%)
Truncating audio 800/1228 (65%)
Truncating audio 900/1228 (73%)
Truncating audio 1000/1228 (81%)
Truncating audio 1100/1228 (90%)
Truncating audio 1200/1228 (98%)
Extracting pauses
Extracting pause spreads


[NeMo W 2023-05-10 19:58:25 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
      ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
    
[NeMo W 2023-05-10 19:58:25 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
      arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
    
[NeMo W 2023-05-10 19:58:25 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
      ret = ret.dtype.type(ret / rcount)
    
[NeMo W 2023-05-10 19:58:25 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
      return _methods._mean(a, axis=axis, dtype=dtype,
    
[NeMo W 2023-05-10 19:58:25 nemo_logging:349] /home/u

Extracting amplitude features
Creating dataframe
Complete
<class 'sklearn.preprocessing._data.MinMaxScaler'>


In [10]:

smile_manager = SmileFeatureManager(data_df)
os_binary_feature_df, os_binary_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='binary')
os_multiclass_feature_df, os_multiclass_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='multiclass')


100%|██████████| 2/2 [00:12<00:00,  6.32s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. duration(seconds)
2. audspec_lengthL1norm_sma_iqr2-3
3. audspec_lengthL1norm_sma_maxPos
4. audspec_lengthL1norm_sma_quartile2
5. audspec_lengthL1norm_sma_range
6. audspec_lengthL1norm_sma_iqr1-2
7. audspec_lengthL1norm_sma_quartile3
8. audspec_lengthL1norm_sma_quartile1
9. audspec_lengthL1norm_sma_minPos
10. audspec_lengthL1norm_sma_iqr1-3
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma[5]_quartile3
2. pcm_fftMag_spectralRollOff90.0_sma_quartile1
3. pcm_fftMag_spectralSkewness_sma_de_posamean
4. mfcc_sma[6]_quartile2
5. pcm_fftMag_spectralRollOff50.0_sma_quartile1
6. pcm_fftMag_spectralEntropy_sma_amean
7. mfcc_sma[2]_amean
8. mfcc_sma[3]_quartile3
9. pcm_fftMag_spectralSkewness_sma_de_lpgain
10. mfcc_sma[3]_quartile2


## Predict and Evaluate

In [11]:
em_model_manager_bin = ModelManager('logreg', em_feature_df, em_feature_cols, merge_train_dev=True)
em_model_manager_bin.trainPredict(label_col='label')

(1.0, 0.1613799612571519)

In [12]:
em_model_manager_multi = ModelManager('logreg', em_feature_df, em_feature_cols, merge_train_dev=True)
em_model_manager_multi.trainPredict(label_col='multiclass_label')

(1.0, 0.1613799612571519)

In [13]:
os_model_manager_bin = ModelManager('logreg', os_binary_feature_df, os_binary_feature_cols, merge_train_dev=True)
os_model_manager_bin.trainPredict(label_col='label')

(0.991869918699187, 0.04902363169122448)

In [14]:
os_model_manager_multi = ModelManager('logreg', os_multiclass_feature_df, os_multiclass_feature_cols, merge_train_dev=True)
os_model_manager_multi.trainPredict(label_col='label')

(1.0, 1.1099331708836112e-05)

In [15]:
cad_model_manager_bin = ModelManager('logreg', cad_feature_df, cad_feature_cols, merge_train_dev=True)
cad_model_manager_bin.trainPredict(label_col='label')

(1.0, 0.05890690830930745)

In [16]:
cad_model_manager_multi = ModelManager('logreg', cad_feature_df, cad_feature_cols, merge_train_dev=True)
cad_model_manager_multi.trainPredict(label_col='multiclass_label')

(1.0, 0.05890690830930745)

In [17]:
print('Binary Performance')
print('---------------------------')
print(f'Embeddings Accuracy: {em_model_manager_bin.accuracy}')
print(f'OpenSmile Accuracy: {os_model_manager_bin.accuracy}')
print(f'Cadence Accuracy: {cad_model_manager_bin.accuracy}')
print()
print(f'Embeddings Loss: {em_model_manager_bin.log_loss_value}')
print(f'OpenSmile Loss: {os_model_manager_bin.log_loss_value}')
print(f'Cadence Loss: {cad_model_manager_bin.log_loss_value}')

Binary Performance
---------------------------
Embeddings Accuracy: 1.0
OpenSmile Accuracy: 0.991869918699187
Cadence Accuracy: 1.0

Embeddings Loss: 0.1613799612571519
OpenSmile Loss: 0.04902363169122448
Cadence Loss: 0.05890690830930745


In [18]:
#am.predCorrelation((em_model_manager_bin, cad_model_manager_bin), label_type='binary')

In [19]:
print('Multi-Class Performance')
print('---------------------------')
print(f'Embeddings Accuracy: {em_model_manager_multi.accuracy}')
print(f'OpenSmile Accuracy: {os_model_manager_multi.accuracy}')
print(f'Cadence Accuracy: {cad_model_manager_multi.accuracy}')
print()
print(f'Embeddings Loss: {em_model_manager_multi.log_loss_value}')
print(f'OpenSmile Loss: {os_model_manager_multi.log_loss_value}')
print(f'Cadence Loss: {cad_model_manager_multi.log_loss_value}')

Multi-Class Performance
---------------------------
Embeddings Accuracy: 1.0
OpenSmile Accuracy: 1.0
Cadence Accuracy: 1.0

Embeddings Loss: 0.1613799612571519
OpenSmile Loss: 1.1099331708836112e-05
Cadence Loss: 0.05890690830930745


In [20]:
#am.predCorrelation((em_model_manager_multi, cad_model_manager_multi), label_type='multiclass')

In [21]:
em_model_manager_bin
os_model_manager_bin
cad_model_manager_bin

In [22]:
import pickle, os

def save_model(model, filepath):
    with open(filepath, 'wb') as file:
        pickle.dump(model, file)

In [23]:
model_dir = '/home/ubuntu/data/demo/models'
save_model(em_model_manager_bin.model, os.path.join(model_dir, 'titanet.pkl'))
save_model(os_model_manager_bin.model, os.path.join(model_dir, 'openSmile.pkl'))
save_model(cad_model_manager_bin.model, os.path.join(model_dir, 'cadence.pkl'))

In [24]:
with open('/home/ubuntu/data/demo/models/openSmile_features.pkl', 'wb') as file:
    pickle.dump(os_binary_feature_cols, file)

In [28]:
with open('/home/ubuntu/data/demo/models/em_features.pkl', 'wb') as file:
    pickle.dump(em_feature_cols, file)

In [25]:
with open('/home/ubuntu/data/demo/models/cadence_scalar.pkl', 'wb') as file:
    pickle.dump(scalar, file)